In [1]:
import numpy as np
import pandas as pd
import psycopg2
from scipy.stats import beta

C:\Users\wojtek\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\wojtek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\wojtek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [22]:
def get_sentiment(stars):
    if stars < 3:
        return -1
    if stars > 3:
        return 1
    return 0

In [6]:
colnames = ['id', 'name', 'city', 'category', 'stars', 'published_at', 'review_text', 'review_id']
df = pd.read_csv('data/319_rest_revs_db.csv', names=colnames, index_col='id', parse_dates=['id'])
df.head()

,name,city,category,stars,published_at,review_text,review_id
id,,,,,,,
0,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2022-06-30 06:24:21.963,NaN,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,Restauracja Zeppelin,Gdańsk,Restaurant,3.0,2020-09-11 18:41:45.985,(Translated by Google) It's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,Restauracja Zeppelin,Gdańsk,Restaurant,4.0,2020-08-23 09:37:34.413,(Translated by Google) Place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-08-20 16:42:43.746,"(Translated by Google) Delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-03-25 21:20:03.338,(Translated by Google) With all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE


In [7]:
df.review_text = df.review_text.fillna('')
df.head()

,name,city,category,stars,published_at,review_text,review_id
id,,,,,,,
0,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2022-06-30 06:24:21.963,,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,Restauracja Zeppelin,Gdańsk,Restaurant,3.0,2020-09-11 18:41:45.985,(Translated by Google) It's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,Restauracja Zeppelin,Gdańsk,Restaurant,4.0,2020-08-23 09:37:34.413,(Translated by Google) Place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-08-20 16:42:43.746,"(Translated by Google) Delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-03-25 21:20:03.338,(Translated by Google) With all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE


In [11]:
df.review_text = df.review_text.fillna('').apply(lambda x: x.lower())
df.head()

,name,city,category,stars,published_at,review_text,review_id
id,,,,,,,
0,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2022-06-30 06:24:21.963,,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,Restauracja Zeppelin,Gdańsk,Restaurant,3.0,2020-09-11 18:41:45.985,(translated by google) it's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,Restauracja Zeppelin,Gdańsk,Restaurant,4.0,2020-08-23 09:37:34.413,(translated by google) place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-08-20 16:42:43.746,"(translated by google) delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-03-25 21:20:03.338,(translated by google) with all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE


In [20]:
df[df.review_text.str.contains('pad thai')]

,name,city,category,stars,published_at,review_text,review_id
id,,,,,,,
2370,AïOLI inspired by Gdańsk,Gdańsk,European restaurant,1.0,2022-06-26 16:13:42.091,(translated by google) every now and then i co...,ChZDSUhNMG9nS0VJQ0FnSURPNkthZVN3EAE
46386,Asia Hoanmy,Gdańsk,Asian restaurant,5.0,2021-07-20 16:41:13.104,(translated by google) i recommend pad thai,ChdDSUhNMG9nS0VJQ0FnSUNhd01URGpBRRAB
46870,Vegan Port,Gdańsk,Vegan restaurant,5.0,2021-11-14 14:45:13.421,(translated by google) very tasty food. carbon...,ChZDSUhNMG9nS0VJQ0FnSUNHXzd1U0t3EAE
47158,PING PONG,Gdańsk,Asian restaurant,5.0,2022-07-02 17:43:58.938,"(translated by google) very tasty, very friend...",ChZDSUhNMG9nS0VJQ0FnSURPcXRfZVdREAE
47167,PING PONG,Gdańsk,Asian restaurant,5.0,2022-06-19 00:44:48.538,najlepszy pad thai jaki jadłam! krewetki w was...,ChZDSUhNMG9nS0VJQ0FnSUNPMDlUdlpnEAE
...,...,...,...,...,...,...,...
96815,Pestka Avocado,Gdańsk,Vegan restaurant,4.0,2022-05-02 06:26:05.802,(translated by google) cool place with vegan f...,ChZDSUhNMG9nS0VJQ0FnSUMyMjR5cU5BEAE
96819,Pestka Avocado,Gdańsk,Vegan restaurant,4.0,2022-04-19 23:02:40.665,"(translated by google) delicious, although a b...",ChdDSUhNMG9nS0VJQ0FnSUMybHJhTTNnRRAB
96824,Pestka Avocado,Gdańsk,Vegan restaurant,5.0,2022-04-13 11:03:12.811,amazing pad thai!,ChZDSUhNMG9nS0VJQ0FnSUMybEtiZkpREAE


In [23]:
# filter by food name
# get a list of name, positive, negative
# sample once from each distr
# use that to rank the list

In [24]:
df['sentiment'] = df.stars.apply(get_sentiment)
df.head()

,name,city,category,stars,published_at,review_text,review_id,sentiment
id,,,,,,,,
0,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2022-06-30 06:24:21.963,,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB,1
1,Restauracja Zeppelin,Gdańsk,Restaurant,3.0,2020-09-11 18:41:45.985,(translated by google) it's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB,0
2,Restauracja Zeppelin,Gdańsk,Restaurant,4.0,2020-08-23 09:37:34.413,(translated by google) place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE,1
3,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-08-20 16:42:43.746,"(translated by google) delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB,1
4,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-03-25 21:20:03.338,(translated by google) with all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE,1


In [25]:
name_df = df[df.review_text.str.contains('dumplings')]

In [61]:
def get_rank(name_df, prior=2, top_res=10):
    rests = []

    for res in name_df.name.unique():
        pos_count = len(name_df[(name_df.name == res) & (name_df.sentiment == 1)])
        neg_count = len(name_df[(name_df.name == res) & (name_df.sentiment == -1)])
        neutral_count = len(name_df[(name_df.name == res) & (name_df.sentiment == 0)])
        # sample from beta distr to get a score
        # for each restaurant with at least one review mentioning the food
        rests.append([
            res,
            pos_count + neg_count + neutral_count,
            f"{pos_count / (pos_count + neg_count + neutral_count):.0%}",
            beta.rvs(prior + pos_count, prior + neg_count)
        ])

    rests.sort(key=lambda x: x[3], reverse=True)
    return rests[:top_res]

In [62]:
get_rank(name_df)

[['Pierogarnia Mandu Gdańsk Śródmieście', 130, '96%', 0.9960852565169344],
 ['Restauracja Gvara', 4, '100%', 0.9636894104467947],
 ['Meso Ramen', 32, '94%', 0.9596127991862633],
 ['Spożywczy Food & Drinks', 12, '100%', 0.9522980650363218],
 ['Food Art Company', 3, '100%', 0.9494168413775609],
 ['Pierogi Lwowskie', 144, '92%', 0.9480698535474141],
 ['House of Seitan', 35, '94%', 0.9445421080230545],
 ['GWAR Bistro', 5, '100%', 0.9243742644841694],
 ['Restauracja KOS', 10, '100%', 0.9156135711816891],
 ['Pan Kotowski', 26, '81%', 0.9117094875330294]]

In [11]:
connection = psycopg2.connect(
   user = 'postgres',
   password = 'my-food-reviews-gdansk',
   host = '34.116.190.191',
   database = 'food-reviews'
)

In [12]:
SQL = 'SELECT * FROM reviews WHERE review_text LIKE %(food)s'

In [13]:
food = 'breakfast'
cur = connection.cursor()
cur.execute(SQL, {'food': f'%{food}%'})

In [17]:
cur.close()
connection.close()

In [14]:
cols = ['id', 'name', 'city', 'category', 'stars', 'published_at', 'review_text', 'review_id']
df = pd.DataFrame(cur.fetchall(), columns=cols)

In [15]:
df.shape

(1677, 8)

In [16]:
df

,id,name,city,category,stars,published_at,review_text,review_id
0,20,Restauracja KOS,Gdańsk,Restaurant,5.0,2022-07-01 09:51:47.833,"(Translated by Google) I recommend, great brea...",ChZDSUhNMG9nS0VJQ0FnSURPNHFmeEtnEAE
1,21,Restauracja KOS,Gdańsk,Restaurant,5.0,2022-07-01 09:49:07.616,(Translated by Google) Very good breakfasts an...,ChZDSUhNMG9nS0VJQ0FnSURPNHNmeWZ3EAE
2,78,Restauracja KOS,Gdańsk,Restaurant,5.0,2022-05-18 20:16:59.461,(Translated by Google) Great place for breakfast.,ChdDSUhNMG9nS0VJQ0FnSUQybVlpYTZBRRAB
3,86,Restauracja KOS,Gdańsk,Restaurant,4.0,2022-05-15 18:20:58.748,"(Translated by Google) Tasty, varied breakfast...",ChRDSUhNMG9nS0VJQ0FnSUQybnJNbxAB
4,94,Restauracja KOS,Gdańsk,Restaurant,5.0,2022-05-08 14:20:10.615,(Translated by Google) MEGA breakfast at MEGA ...,ChZDSUhNMG9nS0VJQ0FnSUQyclBXVU9REAE
...,...,...,...,...,...,...,...,...
1672,97412,Ombrello | Kuchnia Sródziemnomorska,Gdańsk,Mediterranean restaurant,5.0,2021-08-23 08:20:54.397,"(Translated by Google) Great place, good and f...",ChZDSUhNMG9nS0VJQ0FnSURhX3RYQ01REAE
1673,97481,Ombrello | Kuchnia Sródziemnomorska,Gdańsk,Mediterranean restaurant,5.0,2021-07-18 12:02:06.057,(Translated by Google) Delighted by the excell...,ChZDSUhNMG9nS0VJQ0FnSURxdDlmTWFnEAE
1674,97679,Boteco Vieira,Gdańsk,Brazilian restaurant,5.0,2022-04-18 09:28:13.936,(Translated by Google) Delicious breakfast and...,ChZDSUhNMG9nS0VJQ0FnSUMybW9hTkVnEAE
1675,97690,Boteco Vieira,Gdańsk,Brazilian restaurant,5.0,2022-04-02 12:48:00.565,(Translated by Google) Very tasty breakfasts a...,ChZDSUhNMG9nS0VJQ0FnSURXd19YRFJ3EAE


In [23]:
def food_query_db(food):
    connection = psycopg2.connect(
       user = 'postgres',
       password = 'my-food-reviews-gdansk',
       host = '34.116.190.191',
       database = 'food-reviews'
    )
    
    cur = connection.cursor()
    cur.execute(SQL, {'food': f'%{food}%'})
    
    cols = ['id', 'name', 'city', 'category', 'stars', 'published_at', 'review_text', 'review_id']
    df = pd.DataFrame(cur.fetchall(), columns=cols)
    df.published_at = df.published_at.apply(pd.Timestamp)
    df.review_text = df.review_text.fillna('').apply(lambda x: x.lower())

    cur.close()
    connection.close()

    return df


def get_sentiment(stars):
    if stars < 3:
        return -1
    if stars > 3:
        return 1
    return 0


def get_rank(name_df, prior=2, top_res=10):
    name_df['sentiment'] = name_df.stars.apply(get_sentiment)

    rests = []

    for res in name_df.name.unique():
        pos_count = len(name_df[(name_df.name == res) & (name_df.sentiment == 1)])
        neg_count = len(name_df[(name_df.name == res) & (name_df.sentiment == -1)])
        neutral_count = len(name_df[(name_df.name == res) & (name_df.sentiment == 0)])
        # sample from beta distr to get a score
        # for each restaurant with at least one review mentioning the food
        rests.append({
            'name': res,
            'total reviews': pos_count + neg_count + neutral_count,
            '% positive': f"{pos_count / (pos_count + neg_count + neutral_count):.0%}",
            'score': beta.rvs(prior + pos_count, prior + neg_count)
        })

    rests.sort(key=lambda x: x['score'], reverse=True)
    return rests[:top_res]


def get_top_places(food, top_res=10):
    df = food_query_db(food)
    return get_rank(df, top_res=top_res)

In [24]:
get_top_places('spaghetti')

[{'name': 'Classic',
  'total reviews': 1,
  '% positive': '100%',
  'score': 0.9536762097638315},
 {'name': 'Gambas Seafood&Meat',
  'total reviews': 1,
  '% positive': '100%',
  'score': 0.9290964550133974},
 {'name': 'The Culinary Institute of America',
  'total reviews': 1,
  '% positive': '100%',
  'score': 0.9245448000075837},
 {'name': 'Vegan Port',
  'total reviews': 1,
  '% positive': '100%',
  'score': 0.9235598533672233},
 {'name': 'Machina Eats&Beats',
  'total reviews': 7,
  '% positive': '71%',
  'score': 0.8963994376850177},
 {'name': 'Tutti Santi',
  'total reviews': 1,
  '% positive': '100%',
  'score': 0.8793412115880168},
 {'name': 'Restauracja KOS',
  'total reviews': 2,
  '% positive': '100%',
  'score': 0.8744822583727049},
 {'name': 'Jak się masz? Bistro',
  'total reviews': 1,
  '% positive': '0%',
  'score': 0.8692378494317141},
 {'name': 'Ombrello | Kuchnia Sródziemnomorska',
  'total reviews': 5,
  '% positive': '100%',
  'score': 0.8397421989020317},
 {'name